In [40]:
#Get Euribor and Eurirs from sole

In [41]:
url_euribor = "https://mutuionline.24oreborsaonline.ilsole24ore.com/guide-mutui/euribor.asp"
url_eurirs = "https://mutuionline.24oreborsaonline.ilsole24ore.com/guide-mutui/irs.asp"

In [42]:
import requests
import lxml.html as lh
import pandas as pd
import datetime
import numpy as np

In [43]:
def get_proper_size(l):
    
    size = l[0]
    
    for x in l:
        if x != size:
            size = False
            break
    
    return size

In [44]:
def get_df_from_doc(doc):
    tr_elements = doc.xpath('//tr')

    #check if every row is equal
    l = [len(T) for T in tr_elements[:12]]
    rows_ok = get_proper_size(l)

    if rows_ok: #if we got a table

        col = []
        i = 0
        for t in tr_elements[0]:
            i+=1
            name = t.text_content()
            #print(i, name)
            col.append((name, []))

        for j in range(1, len(tr_elements)):
            T = tr_elements[j]
            if len(T) != rows_ok:
                break

            i = 0

            for t in T.iterchildren():
                data = t.text_content()
                col[i][1].append(data)
                i += 1

    cols_ok = get_proper_size([len(C) for (title, C) in col])
    Dict = {title:column for (title, column) in col}
    df = pd.DataFrame(Dict)
    return df

In [45]:
def get_EURIRS_df(doc):
    df = get_df_from_doc(doc)

    #eurirs scraping
    exp_cols = ['Descrizione', 'Fixing', 'Data', 'Fixing Pr.', 'Data Pr.']
    if all(df.columns == exp_cols):
        df.columns = ['Desc', 'Fixing', 'Date', 'Fixing-1', 'Date-1']
        #convert fixing
        df['Fixing'] = [float(x.replace(',', '.').replace('%', ''))/100 for x in df['Fixing']]
        df['Fixing-1'] = [float(x.replace(',', '.').replace('%', ''))/100 for x in df['Fixing-1']]
        #conver date
        df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
        df['Date-1'] = pd.to_datetime(df['Date-1'], format="%d/%m/%Y")
        df['Desc'] = [x.replace("A", "Y") for x in df['Desc']]
        df['Maturity'] = [x.split(' ')[1] for x in df['Desc']]
        df['Settlement'] = 2
    return df

In [46]:
def get_EURIBOR_df(doc):
    df = get_df_from_doc(doc)
    exp_cols = ['Nome', 'Fixing', 'Data', 'Fixing Pr.', 'Data Pr.', 'Fixing 365', 'Fixing 365 Pr.']
    if all(df.columns == exp_cols):
        df.columns = ['Desc', 'Fixing', 'Date', 'Fixing-1', 'Date-1', 'Fixing365', 'Fixing365-1']

        #convert fixing
        df['Fixing'] = [float(x.replace(',', '.').replace('%', ''))/100 for x in df['Fixing']]
        df['Fixing-1'] = [float(x.replace(',', '.').replace('%', ''))/100 for x in df['Fixing-1']]
        df['Fixing365'] = [float(x.replace(',', '.').replace('%', ''))/100 for x in df['Fixing365']]
        df['Fixing365-1'] = [float(x.replace(',', '.').replace('%', ''))/100 for x in df['Fixing365-1']]
        #conver date
        df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
        df['Date-1'] = pd.to_datetime(df['Date-1'], format="%d/%m/%Y")
    return df

In [47]:
#eurirs scraping
page = requests.get(url_eurirs)
doc = lh.fromstring(page.content)
df_EURIRS = get_EURIRS_df(doc)
df_EURIRS

,Desc,Fixing,Date,Fixing-1,Date-1,Maturity,Settlement
0,IRS 1Y,0.0017,2022-04-22,0.0011,2022-04-21,1Y,2
1,IRS 2Y,0.0091,2022-04-22,0.0081,2022-04-21,2Y,2
2,IRS 3Y,0.0121,2022-04-22,0.0111,2022-04-21,3Y,2
3,IRS 4Y,0.0134,2022-04-22,0.0127,2022-04-21,4Y,2
4,IRS 5Y,0.0141,2022-04-22,0.0136,2022-04-21,5Y,2
5,IRS 6Y,0.0148,2022-04-22,0.0142,2022-04-21,6Y,2
6,IRS 7Y,0.0152,2022-04-22,0.0147,2022-04-21,7Y,2
7,IRS 8Y,0.0157,2022-04-22,0.0152,2022-04-21,8Y,2
8,IRS 9Y,0.0162,2022-04-22,0.0158,2022-04-21,9Y,2
9,IRS 10Y,0.0166,2022-04-22,0.0163,2022-04-21,10Y,2


In [48]:
#euribor scraping
page = requests.get(url_euribor)
doc = lh.fromstring(page.content)
df_EURIBOR = get_EURIBOR_df(doc)

In [52]:
#ESTER Scraping
url_ESTER = "https://www.ecb.europa.eu/stats/financial_markets_and_interest_rates/euro_short-term_rate/html/index.en.html"
page = requests.get(url_ESTER)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
tr_elements

ester_fixing = float(tr_elements[0][1].text_content())/100
ester_date = pd.to_datetime(tr_elements[1][1].text_content(), format="%d-%m-%Y")
df_ESTER = pd.DataFrame([['ESTER', ester_fixing, ester_date]])
df_ESTER.columns = ["Desc", "Fixing", "Date"]
df_ESTER['Maturity'] = '1D'#['1d' for x in df_ESTER['Desc']]
df_ESTER['Settlement'] = 0
df_ESTER

,Desc,Fixing,Date,Maturity,Settlement
0,ESTER,-0.00583,2022-04-22,1D,0


In [50]:
#new EURIBOR SCRAPING
url_EURIBOR = "https://www.euribor-rates.eu/en/current-euribor-rates/"
page = requests.get(url_EURIBOR)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
tr_elements[1][1].text_content()

#EURIBOR_date = pd.to_datetime(tr_elements[1][1].text_content(), format="%d-%m-%Y")
labels = ["EURIBOR 1W", "EURIBOR 1M", "EURIBOR 3M", "EURIBOR 6M", "EURIBOR 12M"]
fixing = []
date = pd.to_datetime(tr_elements[0][1].text_content())
for i in range (0, len(labels)):  
    rate = float(tr_elements[i+1][1].text_content().replace(' %', ''))/100
    rate = round(rate, 5)
    fixing.append(rate)
    
df_EURIBOR = pd.DataFrame(list(zip(labels, fixing, [date for x in labels])))
df_EURIBOR.columns = ["Desc", "Fixing", "Date"]
df_EURIBOR['Settlement'] = 2
df_EURIBOR['Maturity'] = [x.split(" ")[1] for x in df_EURIBOR['Desc']]

In [51]:
#last
selection = ['Date', 'Fixing', 'Desc', 'Maturity', 'Settlement']
df_ESTER = df_ESTER[selection]
df_EURIBOR = df_EURIBOR[selection]
df_EURIRS = df_EURIRS[selection]
df_EURIRS

,Date,Fixing,Desc,Maturity,Settlement
0,2022-04-22,0.0017,IRS 1Y,1Y,2
1,2022-04-22,0.0091,IRS 2Y,2Y,2
2,2022-04-22,0.0121,IRS 3Y,3Y,2
3,2022-04-22,0.0134,IRS 4Y,4Y,2
4,2022-04-22,0.0141,IRS 5Y,5Y,2
5,2022-04-22,0.0148,IRS 6Y,6Y,2
6,2022-04-22,0.0152,IRS 7Y,7Y,2
7,2022-04-22,0.0157,IRS 8Y,8Y,2
8,2022-04-22,0.0162,IRS 9Y,9Y,2
9,2022-04-22,0.0166,IRS 10Y,10Y,2
